# Doc2Vec, JUMAN++, KNPで文書類似度算出

## 参考：

- [DeepAge Doc2Vecの仕組みとgensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html#最も似ている記事を取得する)
- [黒橋・河原研究室　自然言語処理のためのリソース](http://nlp.ist.i.kyoto-u.ac.jp/index.php?NLP%E3%83%AA%E3%82%BD%E3%83%BC%E3%82%B9)
- [hassaku's blog 学習済みword2vecモデルを調べてみた](http://blog.hassaku-labs.com/post/pretrained-word2vec/)
- [白ヤギコーポレーション word2vecの学習済み日本語モデルを公開します](http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/)

## インストール

gensim (doc2vec), JUMAN, JUMAN++, KNPを組み込んだDockerを起動

    docker-compose up


## コーパス

livedoorのニュースコーパスをダウンロード

In [1]:
!LDCC=ldcc-20140209 \
 wget http://www.rondhuit.com/download/$LDCC.tar.gz \
 && tar xvfz $LDCC.tar.gz \
 && rm $LDCC.tar.gz

--2017-06-05 15:25:08--  http://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.rondhuit.com/download/ldcc-20140209.tar.gz [following]
--2017-06-05 15:25:08--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: 'ldcc-20140209.tar.gz'

ldcc-20140209.tar.g 100%[===================>]   8.44M   959KB/s    in 13s     

2017-06-05 15:25:22 (661 KB/s) - 'ldcc-20140209.tar.gz' saved [8855190/8855190]

text/
text/CHANGES.txt
text/dokujo-tsushin/
text/it-life-hack/
text/kaden-channel/
text/livedoor-homme/
text/movie-enter/
text/peachy/
text/README.txt
text/smax/
text/sports-watch/
text/

text/topic-news/topic-news-6853845.txt
text/topic-news/topic-news-6855176.txt
text/topic-news/topic-news-6855184.txt
text/topic-news/topic-news-6856653.txt
text/topic-news/topic-news-6856744.txt
text/topic-news/topic-news-6859857.txt
text/topic-news/topic-news-6859873.txt
text/topic-news/topic-news-6860367.txt
text/topic-news/topic-news-6861303.txt
text/topic-news/topic-news-6864824.txt
text/topic-news/topic-news-6866172.txt
text/topic-news/topic-news-6869775.txt
text/topic-news/topic-news-6871429.txt
text/topic-news/topic-news-6873254.txt
text/topic-news/topic-news-6874561.txt
text/topic-news/topic-news-6877206.txt
text/topic-news/topic-news-6877423.txt
text/topic-news/topic-news-6880928.txt
text/topic-news/topic-news-6883210.txt
text/topic-news/topic-news-6886563.txt
text/topic-news/topic-news-6886897.txt
text/topic-news/topic-news-6886968.txt
text/topic-news/topic-news-6893593.txt
text/topic-news/topic-news-6895397.txt
text/topic-news/topic-news-6895782.txt
text/topic-news/topic-new

text/sports-watch/sports-watch-6129622.txt
text/sports-watch/sports-watch-6129709.txt
text/sports-watch/sports-watch-6131308.txt
text/sports-watch/sports-watch-6132879.txt
text/sports-watch/sports-watch-6134059.txt
text/sports-watch/sports-watch-6134071.txt
text/sports-watch/sports-watch-6138700.txt
text/sports-watch/sports-watch-6139403.txt
text/sports-watch/sports-watch-6143201.txt
text/sports-watch/sports-watch-6146894.txt
text/sports-watch/sports-watch-6147140.txt
text/sports-watch/sports-watch-6148681.txt
text/sports-watch/sports-watch-6149027.txt
text/sports-watch/sports-watch-6150974.txt
text/sports-watch/sports-watch-6151095.txt
text/sports-watch/sports-watch-6153054.txt
text/sports-watch/sports-watch-6158040.txt
text/sports-watch/sports-watch-6159841.txt
text/sports-watch/sports-watch-6162603.txt
text/sports-watch/sports-watch-6163537.txt
text/sports-watch/sports-watch-6163542.txt
text/sports-watch/sports-watch-6176333.txt
text/sports-watch/sports-watch-6177641.txt
text/sports

text/smax/smax-6605698.txt
text/smax/smax-6606033.txt
text/smax/smax-6606062.txt
text/smax/smax-6606171.txt
text/smax/smax-6606357.txt
text/smax/smax-6606433.txt
text/smax/smax-6606505.txt
text/smax/smax-6607443.txt
text/smax/smax-6607546.txt
text/smax/smax-6607694.txt
text/smax/smax-6607891.txt
text/smax/smax-6608153.txt
text/smax/smax-6609650.txt
text/smax/smax-6611266.txt
text/smax/smax-6611361.txt
text/smax/smax-6611523.txt
text/smax/smax-6611601.txt
text/smax/smax-6611710.txt
text/smax/smax-6611964.txt
text/smax/smax-6612334.txt
text/smax/smax-6612606.txt
text/smax/smax-6612825.txt
text/smax/smax-6613050.txt
text/smax/smax-6613156.txt
text/smax/smax-6615051.txt
text/smax/smax-6615071.txt
text/smax/smax-6615161.txt
text/smax/smax-6615304.txt
text/smax/smax-6615814.txt
text/smax/smax-6616107.txt
text/smax/smax-6616469.txt
text/smax/smax-6617146.txt
text/smax/smax-6617241.txt
text/smax/smax-6617365.txt
text/smax/smax-6617930.txt
text/smax/smax-6619191.txt
text/smax/smax-6619462.txt
t

text/smax/smax-6781727.txt
text/smax/smax-6781972.txt
text/smax/smax-6782187.txt
text/smax/smax-6782470.txt
text/smax/smax-6782648.txt
text/smax/smax-6782980.txt
text/smax/smax-6783204.txt
text/smax/smax-6784815.txt
text/smax/smax-6784976.txt
text/smax/smax-6785186.txt
text/smax/smax-6785403.txt
text/smax/smax-6785675.txt
text/smax/smax-6786444.txt
text/smax/smax-6786573.txt
text/smax/smax-6786794.txt
text/smax/smax-6788213.txt
text/smax/smax-6788401.txt
text/smax/smax-6788602.txt
text/smax/smax-6788828.txt
text/smax/smax-6789126.txt
text/smax/smax-6789801.txt
text/smax/smax-6789888.txt
text/smax/smax-6790524.txt
text/smax/smax-6790744.txt
text/smax/smax-6791944.txt
text/smax/smax-6792311.txt
text/smax/smax-6792463.txt
text/smax/smax-6792496.txt
text/smax/smax-6792538.txt
text/smax/smax-6792560.txt
text/smax/smax-6792662.txt
text/smax/smax-6792799.txt
text/smax/smax-6792898.txt
text/smax/smax-6793152.txt
text/smax/smax-6793832.txt
text/smax/smax-6794045.txt
text/smax/smax-6794219.txt
t

...

text/kaden-channel/kaden-channel-6395684.txt
text/kaden-channel/kaden-channel-6398774.txt
text/kaden-channel/kaden-channel-6399136.txt
text/kaden-channel/kaden-channel-6400674.txt
text/kaden-channel/kaden-channel-6400675.txt
text/kaden-channel/kaden-channel-6402258.txt
text/kaden-channel/kaden-channel-6403101.txt
text/kaden-channel/kaden-channel-6403750.txt
text/kaden-channel/kaden-channel-6406050.txt
text/kaden-channel/kaden-channel-6406831.txt
text/kaden-channel/kaden-channel-6408146.txt
text/kaden-channel/kaden-channel-6410124.txt
text/kaden-channel/kaden-channel-6410963.txt
text/kaden-channel/kaden-channel-6416150.txt
text/kaden-channel/kaden-channel-6417585.txt
text/kaden-channel/kaden-channel-6422135.txt
text/kaden-channel/kaden-channel-6422527.txt
text/kaden-channel/kaden-channel-6429175.txt
text/kaden-channel/kaden-channel-6431890.txt
text/kaden-channel/kaden-channel-6434638.txt
text/kaden-channel/kaden-channel-6435650.txt
text/kaden-channel/kaden-channel-6436059.txt
text/kaden

text/it-life-hack/it-life-hack-6593130.txt
text/it-life-hack/it-life-hack-6593958.txt
text/it-life-hack/it-life-hack-6594418.txt
text/it-life-hack/it-life-hack-6595464.txt
text/it-life-hack/it-life-hack-6596899.txt
text/it-life-hack/it-life-hack-6598803.txt
text/it-life-hack/it-life-hack-6600578.txt
text/it-life-hack/it-life-hack-6600758.txt
text/it-life-hack/it-life-hack-6601550.txt
text/it-life-hack/it-life-hack-6602191.txt
text/it-life-hack/it-life-hack-6602650.txt
text/it-life-hack/it-life-hack-6604030.txt
text/it-life-hack/it-life-hack-6604237.txt
text/it-life-hack/it-life-hack-6604951.txt
text/it-life-hack/it-life-hack-6605473.txt
text/it-life-hack/it-life-hack-6605921.txt
text/it-life-hack/it-life-hack-6607253.txt
text/it-life-hack/it-life-hack-6607699.txt
text/it-life-hack/it-life-hack-6608171.txt
text/it-life-hack/it-life-hack-6608703.txt
text/it-life-hack/it-life-hack-6610486.txt
text/it-life-hack/it-life-hack-6611530.txt
text/it-life-hack/it-life-hack-6611729.txt
text/it-lif

text/dokujo-tsushin/dokujo-tsushin-4788357.txt
text/dokujo-tsushin/dokujo-tsushin-4788362.txt
text/dokujo-tsushin/dokujo-tsushin-4788373.txt
text/dokujo-tsushin/dokujo-tsushin-4788374.txt
text/dokujo-tsushin/dokujo-tsushin-4788388.txt
text/dokujo-tsushin/dokujo-tsushin-4791665.txt
text/dokujo-tsushin/dokujo-tsushin-4796054.txt
text/dokujo-tsushin/dokujo-tsushin-4799780.txt
text/dokujo-tsushin/dokujo-tsushin-4799892.txt
text/dokujo-tsushin/dokujo-tsushin-4799908.txt
text/dokujo-tsushin/dokujo-tsushin-4799929.txt
text/dokujo-tsushin/dokujo-tsushin-4799933.txt
text/dokujo-tsushin/dokujo-tsushin-4809271.txt
text/dokujo-tsushin/dokujo-tsushin-4809276.txt
text/dokujo-tsushin/dokujo-tsushin-4809277.txt
text/dokujo-tsushin/dokujo-tsushin-4814763.txt
text/dokujo-tsushin/dokujo-tsushin-4814765.txt
text/dokujo-tsushin/dokujo-tsushin-4814778.txt
text/dokujo-tsushin/dokujo-tsushin-4819443.txt
text/dokujo-tsushin/dokujo-tsushin-4826147.txt
text/dokujo-tsushin/dokujo-tsushin-4827888.txt
text/dokujo-t

text/dokujo-tsushin/dokujo-tsushin-5473472.txt
text/dokujo-tsushin/dokujo-tsushin-5474199.txt
text/dokujo-tsushin/dokujo-tsushin-5475979.txt
text/dokujo-tsushin/dokujo-tsushin-5476676.txt
text/dokujo-tsushin/dokujo-tsushin-5478309.txt
text/dokujo-tsushin/dokujo-tsushin-5481232.txt
text/dokujo-tsushin/dokujo-tsushin-5484175.txt
text/dokujo-tsushin/dokujo-tsushin-5486778.txt
text/dokujo-tsushin/dokujo-tsushin-5489562.txt
text/dokujo-tsushin/dokujo-tsushin-5490268.txt
text/dokujo-tsushin/dokujo-tsushin-5492401.txt
text/dokujo-tsushin/dokujo-tsushin-5494962.txt
text/dokujo-tsushin/dokujo-tsushin-5497923.txt
text/dokujo-tsushin/dokujo-tsushin-5500742.txt
text/dokujo-tsushin/dokujo-tsushin-5503526.txt
text/dokujo-tsushin/dokujo-tsushin-5507121.txt
text/dokujo-tsushin/dokujo-tsushin-5507187.txt
text/dokujo-tsushin/dokujo-tsushin-5514666.txt
text/dokujo-tsushin/dokujo-tsushin-5517825.txt
text/dokujo-tsushin/dokujo-tsushin-5521492.txt
text/dokujo-tsushin/dokujo-tsushin-5521634.txt
text/dokujo-t

text/dokujo-tsushin/dokujo-tsushin-6215814.txt
text/dokujo-tsushin/dokujo-tsushin-6217629.txt
text/dokujo-tsushin/dokujo-tsushin-6218046.txt
text/dokujo-tsushin/dokujo-tsushin-6220672.txt
text/dokujo-tsushin/dokujo-tsushin-6223811.txt
text/dokujo-tsushin/dokujo-tsushin-6226941.txt
text/dokujo-tsushin/dokujo-tsushin-6229498.txt
text/dokujo-tsushin/dokujo-tsushin-6230107.txt
text/dokujo-tsushin/dokujo-tsushin-6233611.txt
text/dokujo-tsushin/dokujo-tsushin-6237735.txt
text/dokujo-tsushin/dokujo-tsushin-6242231.txt
text/dokujo-tsushin/dokujo-tsushin-6246330.txt
text/dokujo-tsushin/dokujo-tsushin-6248144.txt
text/dokujo-tsushin/dokujo-tsushin-6250477.txt
text/dokujo-tsushin/dokujo-tsushin-6250970.txt
text/dokujo-tsushin/dokujo-tsushin-6255098.txt
text/dokujo-tsushin/dokujo-tsushin-6255871.txt
text/dokujo-tsushin/dokujo-tsushin-6258044.txt
text/dokujo-tsushin/dokujo-tsushin-6258805.txt
text/dokujo-tsushin/dokujo-tsushin-6261479.txt
text/dokujo-tsushin/dokujo-tsushin-6265258.txt
text/dokujo-t

## 助走

必要ライブラリをimportし、関数をいくつか定義

In [2]:
import sys
from os import listdir, path
from pyknp import Jumanpp
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

記事ファイルをダウンロードしたディレクトリから取得する関数

In [3]:
def corpus_files():
    dirs = [path.join('text', x)
            for x in listdir('text') if not x.endswith('.txt')]
    docs = [path.join(x, y)
            for x in dirs for y in listdir(x) if not x.startswith('LICENSE')]
    return docs

記事コンテンツをパスから取得する関数

In [4]:
def read_document(path):
    with open(path, 'r', encoding="utf-8") as f:
        return f.read()

JUMAN++を使って記事を単語リストに変換する関数

In [5]:
def split_into_words(text):
    result = Jumanpp().analysis(text)
    return [mrph.midasi for mrph in result.mrph_list()]

## 形態素解析

記事コンテンツを単語に分割して、Doc2Vecの入力に使うLabeledSentenceに変換する関数

In [6]:
def doc_to_sentence(doc, name):
    words = split_into_words(doc)
    return LabeledSentence(words=words, tags=[name])

記事のパスリストから、記事コンテンツに変換し、単語分割して、センテンスのジェネレーターを返す関数

In [7]:
def corpus_to_sentences(corpus):
    docs   = [read_document(x) for x in corpus]
    for idx, (doc, name) in enumerate(zip(docs, corpus)):
        sys.stdout.write('\r前処理中 {}/{}'.format(idx, len(corpus)))
        yield doc_to_sentence(doc, name)

Doc2Vecパラメータを渡して、学習

In [ ]:
corpus = corpus_files()
sentences = corpus_to_sentences(corpus)

## 学習

dmに1を設定するとdmpvで学習されることになる。1以外であれば、DBoWで学習される。

In [ ]:
model = Doc2Vec(sentences, dm=0, size=300, window=15, alpha=.025,
        min_alpha=.025, min_count=1, sample=1e-6)

前処理中 7242/7376

In [ ]:
print('\n訓練開始')
for epoch in range(20):
    print('Epoch: {}'.format(epoch + 1))
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= (0.025 - 0.0001) / 19
    model.min_alpha = model.alpha

モデルの保存と読み込み

In [ ]:
!mkdir -p models
model.save('models/doc2vec.model')
model = Doc2Vec.load('models/doc2vec.model')

## 類似記事の取得


最も似ている記事を取得する

In [ ]:
model.docvecs.most_similar('text/livedoor-homme/livedoor-homme-5625149.txt', topn=1)

## 類似度の算出

In [ ]:
model.docvecs.similarity('text/livedoor-homme/livedoor-homme-4700669.txt', 'text/movie-enter/movie-enter-5947726.txt')